# Gateway Router Example

This notebook demonstrates how to use the Gateway router for managing the Gateway container and DEX operations.

The Gateway router provides functionality for:
- Container management (start, stop, status, logs)
- DEX connector configuration
- Network management
- Pool management
- Token discovery

## Setup

Initialize the Hummingbot API client.

In [1]:
from hummingbot_api_client import HummingbotAPIClient

client = HummingbotAPIClient(base_url='http://localhost:8000')
await client.init()

## Container Management

Manage the Gateway Docker container.

In [2]:
# Get Gateway container status
status = await client.gateway.get_status()
status

{'running': True,
 'container_id': 'aa479985457937dfa73d0a527e9b16ae87382152adc864971089ab3afb9c993e',
 'image': 'hummingbot/gateway:latest',
 'created_at': '2025-10-17T14:16:20.740872044Z',
 'port': 15888}

In [41]:
# Start Gateway container (configure as needed)
config = {
    "image": "hummingbot/gateway:latest",
    "port": 15888,
    "passphrase": "a",
    "environment": {}
}
result = await client.gateway.start(config)
result

{'success': True,
 'message': 'Gateway started successfully',
 'container_id': 'dc3041ce9b0e5bb55081d047b41a178d15745b9ff683d120ddcc285ebb9f8d98',
 'port': 15888}

In [32]:
# Get Gateway logs (last 50 lines)
logs = await client.gateway.get_logs(tail=50)
print(logs.get('logs', ''))

2025-10-17T14:37:51.939461836Z   "error": {},
2025-10-17T14:37:51.939477003Z   "rejection": true,
2025-10-17T14:37:51.939479170Z   "date": "Fri Oct 17 2025 14:37:51 GMT+0000 (Coordinated Universal Time)",
2025-10-17T14:37:51.939480878Z   "process": {
2025-10-17T14:37:51.939482336Z     "pid": 20,
2025-10-17T14:37:51.939483711Z     "uid": 0,
2025-10-17T14:37:51.939518503Z     "gid": 0,
2025-10-17T14:37:51.939522461Z     "cwd": "/home/gateway",
2025-10-17T14:37:51.939524211Z     "execPath": "/usr/local/bin/node",
2025-10-17T14:37:51.939525586Z     "version": "v20.19.4",
2025-10-17T14:37:51.939526961Z     "argv": [
2025-10-17T14:37:51.939528211Z       "/usr/local/bin/node",
2025-10-17T14:37:51.939529420Z       "/home/gateway/dist/index.js",
2025-10-17T14:37:51.939530628Z       "--dev"
2025-10-17T14:37:51.939531795Z     ],
2025-10-17T14:37:51.939532878Z     "memoryUsage": {
2025-10-17T14:37:51.939534086Z       "rss": 156086272,
2025-10-17T14:37:51.939535253Z       "heapTotal": 91389952,
202

In [40]:
# Stop Gateway container
result = await client.gateway.stop()
result

{'success': True, 'message': 'Gateway stopped successfully'}

## DEX Connectors

List and configure DEX connectors.

In [7]:
# List all available DEX connectors
connectors = await client.gateway.list_connectors()
connectors

{'connectors': [{'name': 'jupiter',
   'trading_types': ['router'],
   'chain': 'solana',
   'networks': ['devnet', 'mainnet-beta']},
  {'name': 'meteora',
   'trading_types': ['clmm'],
   'chain': 'solana',
   'networks': ['devnet', 'mainnet-beta']},
  {'name': 'raydium',
   'trading_types': ['amm', 'clmm'],
   'chain': 'solana',
   'networks': ['devnet', 'mainnet-beta']},
  {'name': 'uniswap',
   'trading_types': ['amm', 'clmm', 'router'],
   'chain': 'ethereum',
   'networks': ['arbitrum',
    'avalanche',
    'base',
    'bsc',
    'celo',
    'mainnet',
    'optimism',
    'polygon']},
  {'name': '0x',
   'trading_types': ['router'],
   'chain': 'ethereum',
   'networks': ['arbitrum',
    'avalanche',
    'base',
    'bsc',
    'mainnet',
    'optimism',
    'polygon']}]}

In [11]:
# Get configuration for a specific connector (e.g., Meteora)
config = await client.gateway.get_connector_config('meteora')
config

{'slippage_pct': 1, 'strategy_type': 0}

In [12]:
# Update connector configuration (snake_case or camelCase)
updates = {
    "slippage_pct": 1.0,  # 0.5% slippage
    # or use camelCase: "slippagePct": 0.5
}
result = await client.gateway.update_connector_config('meteora', updates)
result

{'success': True,
 'message': 'Updated 1 config parameter(s) for meteora. Restart Gateway for changes to take effect.',
 'restart_required': True,
 'restart_endpoint': 'POST /gateway/restart',
 'results': [{'message': "Configuration updated successfully: 'meteora.slippagePct' set to 1"}]}

## Chains and Networks

Discover available blockchain networks.

In [13]:
# List all chains and their networks
chains = await client.gateway.list_chains()
chains

{'chains': [{'chain': 'ethereum',
   'networks': ['arbitrum',
    'avalanche',
    'base',
    'bsc',
    'celo',
    'mainnet',
    'optimism',
    'polygon',
    'sepolia']},
  {'chain': 'solana', 'networks': ['devnet', 'mainnet-beta']}]}

In [14]:
# List all networks (flattened view)
networks = await client.gateway.list_networks()
networks

{'networks': [{'network_id': 'ethereum-arbitrum',
   'chain': 'ethereum',
   'network': 'arbitrum'},
  {'network_id': 'ethereum-avalanche',
   'chain': 'ethereum',
   'network': 'avalanche'},
  {'network_id': 'ethereum-base', 'chain': 'ethereum', 'network': 'base'},
  {'network_id': 'ethereum-bsc', 'chain': 'ethereum', 'network': 'bsc'},
  {'network_id': 'ethereum-celo', 'chain': 'ethereum', 'network': 'celo'},
  {'network_id': 'ethereum-mainnet',
   'chain': 'ethereum',
   'network': 'mainnet'},
  {'network_id': 'ethereum-optimism',
   'chain': 'ethereum',
   'network': 'optimism'},
  {'network_id': 'ethereum-polygon',
   'chain': 'ethereum',
   'network': 'polygon'},
  {'network_id': 'ethereum-sepolia',
   'chain': 'ethereum',
   'network': 'sepolia'},
  {'network_id': 'solana-devnet', 'chain': 'solana', 'network': 'devnet'},
  {'network_id': 'solana-mainnet-beta',
   'chain': 'solana',
   'network': 'mainnet-beta'}],
 'count': 11}

## Network Configuration

Configure specific networks (e.g., RPC endpoints).

In [42]:
# Get network configuration (format: chain-network)
config = await client.gateway.get_network_config('solana-mainnet-beta')
config

{'node_url': 'https://mainnet.helius-rpc.com/?api-key=a43dcfb5-df0c-45e6-b3d8-6091d0d846dd',
 'native_currency_symbol': 'SOL',
 'default_compute_units': 250000,
 'confirm_retry_interval': 0.5,
 'confirm_retry_count': 15,
 'base_priority_fee_pct': 90,
 'min_priority_fee_per_cu': 0.1}

In [39]:
# Update network configuration
updates = {
    "node_url": "https://api.mainnet-beta.solana.com",
    # or use camelCase: "nodeURL": "https://..."
}
result = await client.gateway.update_network_config('solana-mainnet-beta', updates)
result

{'success': True,
 'message': 'Updated 1 config parameter(s) for solana-mainnet-beta. Restart Gateway for changes to take effect.',
 'restart_required': True,
 'restart_endpoint': 'POST /gateway/restart',
 'results': [{'message': 'Configuration updated successfully: \'solana-mainnet-beta.nodeURL\' set to "https://mainnet.helius-rpc.com/?api-key=a43dcfb5-df0c-45e6-b3d8-6091d0d846dd"'}]}

## Token Discovery

Find available tokens on a network.

In [17]:
# Get all tokens for a network
tokens = await client.gateway.get_network_tokens('solana-mainnet-beta')
tokens

{'tokens': [{'name': 'Bonk',
   'symbol': 'BONK',
   'address': 'DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7YaB1pPB263',
   'decimals': 5},
  {'name': 'Ethereum (Wormhole)',
   'symbol': 'ETH',
   'address': '7vfCXTUXx5WJV5JADk17DUJ4ksgau7utNKj4b963voxs',
   'decimals': 8},
  {'name': 'Helium',
   'symbol': 'HNT',
   'address': 'hntyVP6YFm1Hg25TN9WGLqM12b8TQmcknKrdu1oxWux',
   'decimals': 8},
  {'name': 'Jupiter',
   'symbol': 'JUP',
   'address': 'JUPyiwrYJFskUPiHa7hkeR8VUtAeFoSYbKedZNsDvCN',
   'decimals': 6},
  {'name': 'Marinade Staked SOL',
   'symbol': 'MSOL',
   'address': 'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So',
   'decimals': 9},
  {'name': 'Pyth Network',
   'symbol': 'PYTH',
   'address': 'HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3RKwX8eACQBCt3',
   'decimals': 6},
  {'name': 'Raydium',
   'symbol': 'RAY',
   'address': '4k3Dyjzvzp8eMZWUXbBCjEvwSkkk59S5iCNLY3QrkX6R',
   'decimals': 6},
  {'name': 'Wrapped Solana',
   'symbol': 'SOL',
   'address': 'So11111111111111111111111111111111111111

In [18]:
# Search for specific tokens (e.g., USDC)
usdc_tokens = await client.gateway.get_network_tokens('solana-mainnet-beta', search='USDC')
usdc_tokens

{'tokens': [{'name': 'USDC',
   'symbol': 'USDC',
   'address': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
   'decimals': 6}]}

### Add Custom Tokens

You can add custom tokens that are not in the default token list.

In [ ]:
# Add Fartcoin token to Solana mainnet-beta
result = await client.gateway.add_token(
    network_id='solana-mainnet-beta',
    address='9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump',
    symbol='FARTCOIN',
    decimals=6,  # Check token decimals on-chain
    name='Fartcoin'
)
result

In [ ]:
# Verify the token was added by searching for it
fartcoin_tokens = await client.gateway.get_network_tokens('solana-mainnet-beta', search='FARTCOIN')
fartcoin_tokens

### Delete Custom Tokens

Remove tokens that were previously added.

**Note:** Token deletion may require the Gateway to support this feature. If you get a "Validation Error", it means the underlying Gateway client doesn't have the delete_token method implemented yet.

In [ ]:
# Delete the Fartcoin token we added earlier
result = await client.gateway.delete_token(
    network_id='solana-mainnet-beta',
    token_address='9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump'
)
result

In [ ]:
# Verify the token was deleted by searching for it
fartcoin_tokens_after_delete = await client.gateway.get_network_tokens('solana-mainnet-beta', search='FARTCOIN')
print(f"FARTCOIN tokens found: {len(fartcoin_tokens_after_delete.get('tokens', []))}")
fartcoin_tokens_after_delete

## Liquidity Pools

List and manage liquidity pools for DEX connectors.

In [28]:
# List pools for a connector and network
pools = await client.gateway.list_pools(
    connector_name='meteora',
    network='mainnet-beta'
)
pools

[{'type': 'clmm',
  'network': 'mainnet-beta',
  'base': 'SOL',
  'quote': 'USDC',
  'address': '2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3',
  'trading_pair': 'SOL-USDC'},
 {'type': 'clmm',
  'network': 'mainnet-beta',
  'base': 'TRUMP',
  'quote': 'USDC',
  'address': '9d9mb8kooFfaD3SctgZtkxQypkshx6ezhbKio89ixyy2',
  'trading_pair': 'TRUMP-USDC'},
 {'type': 'clmm',
  'network': 'mainnet-beta',
  'base': 'JUP',
  'quote': 'USDC',
  'address': '7HR1ouGwPsCyPScUCx4WJCPyktXMoLitrGkLczd7Vabx',
  'trading_pair': 'JUP-USDC'}]